# **Loan Default Prediction**

This notebook demonstrates training a **PyTorch neural network** to predict whether a loan will default.

## **Notebook Outline**
1. **Train-Test Split & Imbalance Handling**
2. **PyTorch Model Building & Training**
3. **Evaluation & Threshold Tuning**

We'll see both **Markdown explanations** (like this one) and **Code cells** demonstrating each step.

---

In [1]:
# (Cell 1) 1. Imports & Setup
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, 
                             f1_score, roc_auc_score, classification_report)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import RandomOverSampler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

plt.style.use('seaborn')  # optional aesthetics
print("Setup complete.")

Setup complete.


<ipython-input-1-7137d4bfe437>:23: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')  # optional aesthetics


## **Cleaned Data Loading For Traning**
We load the "cleaned_train_data.csv" file that we saved after EDA analysis for training.

In [2]:
df = pd.read_csv("cleaned_train_data.csv")


In [3]:
df 

,loan_amnt,term,int_rate,percent_bc_gt_75,bc_util,dti,inq_last_6mths,revol_util,mths_since_last_major_derog,internal_score,...,home_ownership_RENT,purpose_debt_consolidation,purpose_house,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,7550,36.0,16.24,100.0,96.0,8.40,0.0,72.0,999.0,99,...,True,True,False,False,False,False,False,False,False,False
1,27050,36.0,10.99,25.0,53.9,22.87,0.0,61.2,999.0,353,...,False,True,False,False,False,False,False,False,False,False
2,12000,36.0,10.99,0.0,15.9,4.62,1.0,24.0,999.0,157,...,True,True,False,False,False,False,False,False,False,False
3,28000,36.0,7.62,16.7,67.1,18.55,1.0,54.6,999.0,365,...,False,True,False,False,False,False,False,False,False,False
4,12000,36.0,13.53,33.3,79.6,16.94,0.0,68.8,53.0,157,...,True,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189452,10025,36.0,15.88,100.0,100.1,16.99,0.0,80.3,999.0,131,...,False,False,False,False,False,False,False,False,False,False
189453,16000,60.0,14.33,25.0,66.4,13.96,3.0,66.4,17.0,209,...,False,False,False,False,False,False,False,False,False,False
189454,16000,60.0,16.78,100.0,97.1,14.58,0.0,97.1,999.0,209,...,False,True,False,False,False,False,False,False,False,False
189455,6000,36.0,19.52,0.0,0.0,13.77,1.0,47.9,999.0,79,...,True,False,False,True,False,False,False,False,False,False


## **6. Train-Test Split & Imbalance Handling**

We'll do a stratified split (to preserve the ~7% minority class). Then we optionally oversample the minority class using `RandomOverSampler` from `imblearn`.

In [4]:
# (Cell 5) Train-Val Split, Scale, and Oversample

y = df['bad_flag'].astype(int)
X = df.drop(columns=['bad_flag'])

# Stratified split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape, "Val shape:", X_val.shape)
print("Positive class in train:", (y_train==1).mean())
print("Positive class in val:  ", (y_val==1).mean())

# Scale numeric features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

# Convert back to DataFrame (optional)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val   = pd.DataFrame(X_val_scaled,   columns=X_val.columns,   index=X_val.index)

# OverSampling
ros = RandomOverSampler(random_state=42, sampling_strategy=0.15)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)
print("After oversampling:", X_train_ros.shape)
print("Positive class proportion:", (y_train_ros==1).mean())

Train shape: (151565, 20) Val shape: (37892, 20)
Positive class in train: 0.06929040345726256
Positive class in val:   0.06930222738308878
After oversampling: (162222, 20)
Positive class proportion: 0.13043237045530198


## **7. PyTorch Model Building & Training**
We'll define a **DeeperNet** with a couple of hidden layers. We'll use a **pos_weight** in the BCE loss to handle imbalance. Then train for a set number of epochs.

**Why this architecture?**
- A 2-layer MLP is more expressive than a single-layer and still relatively fast.
- We use ReLU activation for simplicity.
- We use Adam optimizer with a small learning rate. In practice, you might tune more extensively.


In [5]:
# (Cell 6) Build PyTorch model & Dataloaders

# Prepare final train sets
use_oversample = True  # set to False if you want to skip oversampling

if use_oversample:
    X_train_final, y_train_final = X_train_ros, y_train_ros
else:
    X_train_final, y_train_final = X_train, y_train

X_train_t = torch.tensor(X_train_final.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train_final.values, dtype=torch.float32)

X_val_t = torch.tensor(X_val.values, dtype=torch.float32)
y_val_t = torch.tensor(y_val.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset   = TensorDataset(X_val_t,   y_val_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=64, shuffle=False)

class DeeperNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # no sigmoid here for BCEWithLogitsLoss
        return x

# Initialize the network
model = DeeperNet(input_dim=X_train.shape[1], hidden_dim=64)
print(model)

# pos_weight for imbalance
neg_count = (y_train == 0).sum()
pos_count = (y_train == 1).sum()
pos_weight_val = torch.tensor([neg_count / pos_count], dtype=torch.float32)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_val)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    correct, total = 0, 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Compute train accuracy
        probs = torch.sigmoid(outputs)
        preds = (probs > 0.5).float()
        correct += (preds == batch_y).sum().item()
        total   += batch_y.size(0)

    train_acc = correct / total
    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for val_X, val_y in val_loader:
            val_outputs = model(val_X).squeeze()
            v_loss = criterion(val_outputs, val_y)
            val_loss += v_loss.item()

            val_probs = torch.sigmoid(val_outputs)
            val_preds = (val_probs > 0.5).float()
            val_correct += (val_preds == val_y).sum().item()
            val_total   += val_y.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total

    if (epoch+1) % 1 == 0:
        print(f"[Epoch {epoch+1}/{epochs}] "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")


DeeperNet(
  (fc1): Linear(in_features=20, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)
[Epoch 1/50] Train Loss: 1.5675, Train Acc: 0.3132 | Val Loss: 1.2783, Val Acc: 0.3124
[Epoch 2/50] Train Loss: 1.5524, Train Acc: 0.3318 | Val Loss: 1.2899, Val Acc: 0.3062
[Epoch 3/50] Train Loss: 1.5463, Train Acc: 0.3395 | Val Loss: 1.3365, Val Acc: 0.2721
[Epoch 4/50] Train Loss: 1.5406, Train Acc: 0.3407 | Val Loss: 1.2772, Val Acc: 0.3270
[Epoch 5/50] Train Loss: 1.5352, Train Acc: 0.3446 | Val Loss: 1.2780, Val Acc: 0.3516
[Epoch 6/50] Train Loss: 1.5300, Train Acc: 0.3511 | Val Loss: 1.2834, Val Acc: 0.3374
[Epoch 7/50] Train Loss: 1.5252, Train Acc: 0.3577 | Val Loss: 1.2795, Val Acc: 0.3441
[Epoch 8/50] Train Loss: 1.5198, Train Acc: 0.3601 | Val Loss: 1.3141, Val Acc: 0.2994
[Epoch 9/50] Train Loss: 1.5147, Train Acc: 0.3659 | Val Loss: 1.2789, Val Acc: 0.3400
[Epoch 10/50] Train Loss:

## **8. Evaluation & Threshold Tuning**
We'll compute **precision, recall, F1, and AUC** on the validation set. Then we can see if a threshold other than 0.5 yields better F1.

In [6]:
# (Cell 7) Final Evaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for val_X, val_y in val_loader:
        logits = model(val_X).squeeze()
        probs = torch.sigmoid(logits)
        all_preds.extend(probs.cpu().numpy())
        all_labels.extend(val_y.cpu().numpy())

# Default threshold = 0.5
pred_labels = [1 if p >= 0.65 else 0 for p in all_preds]

prec = precision_score(all_labels, pred_labels)
rec  = recall_score(all_labels, pred_labels)
f1   = f1_score(all_labels, pred_labels)
auc  = roc_auc_score(all_labels, all_preds)

print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")

# Classification report
print("\nClassification Report:\n", classification_report(all_labels, pred_labels, target_names=["class 0", "class 1"]))

Precision: 0.0962, Recall: 0.6181, F1: 0.1665, AUC: 0.6298

Classification Report:
               precision    recall  f1-score   support

     class 0       0.95      0.57      0.71     35266
     class 1       0.10      0.62      0.17      2626

    accuracy                           0.57     37892
   macro avg       0.52      0.59      0.44     37892
weighted avg       0.89      0.57      0.67     37892



### Threshold Tuning
To find the best F1 threshold, we can systematically try thresholds from 0.0 to 1.0 in small increments. This is just a demonstration—pick a threshold that aligns with your business needs (precision vs. recall).

In [7]:
# (Cell 8) Threshold Tuning
best_t, best_f1 = 0, 0
import numpy as np

for t in np.arange(0.0, 1.01, 0.01):
    temp_preds = [1 if p >= t else 0 for p in all_preds]
    current_f1 = f1_score(all_labels, temp_preds)
    if current_f1 > best_f1:
        best_f1 = current_f1
        best_t = t

print("Best threshold:", best_t)
print("Best F1 at that threshold:", best_f1)

Best threshold: 0.74
Best F1 at that threshold: 0.17567665102850955


## **8. Data Alignment and Scaling for Test Set**
This section ensures that the test dataset is properly aligned with the features used during model training. Missing columns are added with default values, and any unnecessary columns are removed. After alignment, the test data is scaled using the same scaler fitted on the training data to maintain consistency in feature scaling. Then we do the inference to do the **loan default prediction**.


In [8]:
import pandas as pd

# Load test data
test_df = pd.read_csv("testing_loan_data.csv")


<ipython-input-8-aab07725946a>:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("testing_loan_data.csv")


In [9]:
# 1. Convert 'term' from "36 months" → 36
if 'term' in test_df.columns:
    test_df['term'] = test_df['term'].str.replace(' months', '', regex=False)
    test_df['term'] = pd.to_numeric(test_df['term'], errors='coerce')

# 2. Convert percentage columns like 'int_rate' and 'revol_util'
for col in ['int_rate', 'revol_util']:
    if col in test_df.columns:
        test_df[col] = test_df[col].astype(str).str.replace('%', '', regex=False)
        test_df[col] = pd.to_numeric(test_df[col], errors='coerce')

# 3. Convert 'emp_length' to numeric
def convert_emp_length(x):
    if pd.isnull(x):
        return None
    x = str(x).lower()
    if '10+' in x:
        return 10
    elif '< 1' in x:
        return 0
    match = re.search(r'(\d+)', x)
    return int(match.group(1)) if match else None

if 'emp_length' in test_df.columns:
    test_df['emp_length'] = test_df['emp_length'].apply(convert_emp_length)

# 4. Handle missing values
test_df['mths_since_last_major_derog'].fillna(999, inplace=True)
test_df['mths_since_recent_inq'].fillna(36, inplace=True)

num_impute_cols = [
    'tot_cur_bal', 'bc_util', 'percent_bc_gt_75', 'emp_length',
    'tot_hi_cred_lim', 'total_bc_limit', 'revol_util', 'dti',
    'inq_last_6mths', 'annual_inc', 'int_rate', 'term'
]
for col in num_impute_cols:
    test_df[col].fillna(test_df[col].median(), inplace=True)

# 5. One-hot encode categorical columns
categorical_cols = ['home_ownership', 'purpose']
test_df = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)


In [10]:
# Drop low-impact columns based on correlation analysis
columns_to_drop = [
    'home_ownership_OTHER', 'home_ownership_NONE', 'home_ownership_OWN',
    'purpose_major_purchase', 'purpose_home_improvement', 'mths_since_recent_inq',
    'emp_length', 'purpose_credit_card', 'annual_inc', 'tot_cur_bal',
    'tot_hi_cred_lim', 'total_bc_limit'
]

test_df = test_df.drop(columns=columns_to_drop, errors='ignore')


In [11]:
# Load the columns from the training set
trained_columns = X_train.columns.tolist()

# Drop unnecessary columns in test data
test_df_cleaned = test_df.drop(columns=['desc', 'id', 'member_id', 'application_approved_flag', 'bad_flag'], errors='ignore')

# Add missing columns with default values (0 for one-hot encoded columns)
for col in trained_columns:
    if col not in test_df_cleaned.columns:
        test_df_cleaned[col] = 0

# Ensure test data columns are in the same order as training
X_test = test_df_cleaned[trained_columns]


In [12]:
from sklearn.preprocessing import StandardScaler
 # Ensure columns match training features
X_test_scaled = scaler.transform(X_test)  # Scale the data


In [13]:
# Convert test data to PyTorch tensor
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Make predictions
with torch.no_grad():
    logits = model(X_test_tensor).squeeze()
    probs = torch.sigmoid(logits)  # Convert logits to probabilities

# Convert probabilities to binary predictions
threshold = 0.70
pred_labels = (probs >= threshold).int().numpy()



In [14]:
# Save predictions
test_df['bad_flag'] = pred_labels
test_df[['id', 'bad_flag']].to_csv("test_predictions.csv", index=False)
